# Firework Algorithm for Optimization

- Autores originais: [*Ying Tan*](https://scholar.google.com/citations?user=PjNxSPsAAAAJ&hl=en) e [*Yuanchun Zhu*](https://ieeexplore.ieee.org/author/38008573100)
- Ano de Publicação: 2010
- Editora: [Springer](https://link.springer.com/chapter/10.1007/978-3-642-13495-1_44)
- Conferência: [Advances in Swarm Intelligence](https://link.springer.com/book/10.1007/978-3-642-13495-1): First International Conference, ICSI 2010, Beijing, China, June 12-15, 2010, Proceedings, Part I 1

## Descrição:
FA é um algoritmo de inteligência de enxames inspirado na explosão de fogos de artifício. Esse algoritmo é utilizado para a otimização global de funções complexas. São fornecidos dois processos de *“explosão”* (busca) e os mecanismos para manter a diversidade das *“faíscas”*.

In [1]:
import numpy as np
import pandas as pd
import random

## Funções Objetivos

In [2]:
def sphere_function(x):
    result = x**2
    return np.sum(result)

In [3]:
def rastrigin(x):
    n = len(x)
    total = 0
    total = x**2 - 10*np.cos(2*np.pi*x)
    return (10*n) + total

In [4]:
def rosenbrook(x):
    n = len(x)
    term1 = 100 * (x[1:] - x[:-1]**2)**2
    temr2 = (x[1:] -1)**2
    sumterm = np.sum(term1 + term2)
    return sumterm

In [5]:
def ackley(x):
    n = len(x)
    sum1 = np.sum(x*2)
    sum2 = np.sum(np.cos(2 * np.pi * x**2))
    term1 = -20 * np.exp(-0.2 * np.sqrt(1/n * sum1))
    term2 = -np.exp(1/n * sum2)
    return term1 + term2 + 20 + np.exp(1)

### Distribuição Gaussiana:

In [6]:
value = random.gauss(1,1)
value

0.024486290735798777

### Distribuição Uniforme:

In [7]:
value = random.uniform(-1,1)
value

0.5859760883579517

## Inicializando posições dos Fogos de Artifício:

In [8]:
def initialize_firework_locations(n, d, xmin, xmax):
    fireworks = xmin + (xmax - xmin) * np.random.rand(n,d)
    return fireworks

## Calculando o número de faíscas:

In [9]:
def sparks_number(fx, fX, m, ymax, eps=.00001):
    sum_difference_of_worst_fit_and_fx = np.sum(ymax-fX) + eps
    difference_of_max_and_fx = (ymax - fx + eps)
    s = m * difference_of_max_and_fx/sum_difference_of_worst_fit_and_fx
    s = firework_boundaries(0.1, 0.8, m, s)
    return s

In [10]:
def firework_boundaries(a, b, m, s):
    term1 = np.round(a*m) * (s < (a*m))
    term2 = np.round(b*m) * (s> b*m)
    term3 = np.round(s) * (~(s < (a*m)) & ~((s > b *m) & (a<b<1)))
    s_new = term1 + term2 + term3
    return s_new

## Obtendo localização das Faíscas:

### Seletor de dimensões aleatório

In [11]:
def random_dimension_selector(d):
    xj = np.zeros(d)
    z = round(d * random.uniform(0,1))
    random_dimensions = np.random.choice(d, z, replace=False)
    for i in random_dimensions:
        xj[i] = 1
    return xj
    

### Amplitude de explosão

In [12]:
def explosion_amplitude(A_hat, fx,fX, ymin, eps=.00001):
    sum_difference_of_fx_and_best_fit = np.sum(fX - ymin) + eps
    difference_of_fx_and_min = (fx - ymin + eps)
    A = A_hat * difference_of_fx_and_min/sum_difference_of_fx_and_best_fit
    return A

In [13]:
d = 6
round(6 * random.uniform(0,1))

2

### Obtendo faísca: Estratégia da Explosão

In [14]:
def obtain_location_of_spark(d, amp, xmin, xmax, xi):
    #selecting z dimensions to be affected
    xj = xi
    random_selected_positions = random_dimension_selector(d) 
    h = amp * random.uniform(-1 ,1) 
    random_affected_dimensions = xj + (random_selected_positions* h)
    #mapping affected dimensions to potential space
    random_affected_dimensions = map_to_potential_space(xmin, xmax, random_affected_dimensions)
    return random_affected_dimensions

### Mapeamento para espaço potencial:

In [15]:
def map_to_potential_space(xmin, xmax, affected):
    inbound = xmin + abs(affected)%(xmax - xmin)
    term1 = (affected > xmax) * inbound
    term2 =  (affected < xmin) * inbound
    fixed_bounds = term1 + term2
    mapping = (fixed_bounds == 0) * affected + fixed_bounds 
    return mapping

In [16]:
xi = np.random.rand(3)
xmin = np.array([-3.12, -4.5, -6.6])
xmax = np.array([3.12, 4.5, 6.6])
d = 3
amp = 5
obtain_location_of_spark(d, amp, xmin, xmax, xi)

array([0.69752719, 0.69360941, 0.96435884])

### Obtendo faísca: Estratégia de Faíscas Gaussianas:

In [17]:
def gaussian_spark_location(xmin, xmax, d, xi):
    xj = xi
    random_dimensions_selected = random_dimension_selector(d)
    random_affected_dimensions = (xj * np.logical_not(random_dimensions_selected)) + xj * (random_dimensions_selected * np.random.normal(1,1,d))
    random_affected_dimensions = map_to_potential_space(xmin, xmax, random_affected_dimensions)
    return random_affected_dimensions

In [18]:
d =3
np.random.normal(1,1,d)

array([ 0.24729605, -0.31566698,  1.89294813])

In [19]:
np.random.rand(3)

array([0.34749793, 0.58474792, 0.28910127])

## Seleção de Localizações:

### Calculando Distância Geral:

In [20]:
def general_distance(xi , X, d):
    sum_of_distances = np.sum(abs(xi - X))
    return sum_of_distances

### Calculando probabilidade de seleção:

In [21]:
def location_probability(RX, Rxi,d):
    pxi = Rxi/RX
    return pxi

## Framework ALgoritmo de Otimização Fogos de Artifício:

In [22]:
#localizações
n = 10
#quantidade de faíscas
m = 50
#dimensões
d = 10
#parâmetro para amplitude
a = 20
#faíscas gaussianas
gauss = 15

#Número de fogos na estratégia gaussiana
m_hat = 5

#minimo valor do espaço potencial
xmin = -5

#máximo valor do espaço potencial
xmax = 5

#Inicializando fogoso de artifício
X = initialize_firework_locations(n, d, xmin, xmax)

#Total de iterações
explosion_generations = 10

current_generation = 0
while(current_generation < explosion_generations):
        #f(X)
    y = np.zeros((n,1))
    for i in range(n):
        y[i] = sphere_function(X[i])

    #best fit
    ymin = y.min()
    #worst fit
    ymax = y.max()

    #Numero de faíscas por fogo de artifício
    S = np.zeros((n,1))
    #Amplitude de cada fogo de artificio
    A = np.zeros((n,1))

    for i in range(n):
        S[i] = sparks_number(y[i],y, m, ymax) 
        A[i] = explosion_amplitude(a, y[i],y, ymin)

    total_sparks = np.sum(S)
    new_locations = X
    generation = np.array((n,d))
    for i in range(n):
        xi = X[i]
        n_sparks = int(S[i][0])
        sparks = np.zeros((n_sparks,d))
        for j in range(n_sparks):
            sparks[j] = obtain_location_of_spark(d, a, xmin, xmax, xi)
        new_locations = np.vstack((new_locations, sparks)) 

    random_fireworks = np.sort(np.random.choice(n,m_hat, replace=False))
    for i in range(m_hat):
        xi = X[random_fireworks[i]]
        gaussian_sparks = np.zeros((gauss,d))
        for j in range(gauss):
            gaussian_sparks[j] = gaussian_spark_location(xmin, xmax, d, xi)
        new_locations = np.vstack((new_locations, gaussian_sparks)) 
    total_locations = int(total_sparks) + n + m_hat*gauss
    fx_new_locations = np.zeros((total_locations, 1))
    for i in range(total_locations):
        fx_new_locations[i] = sphere_function(new_locations[i])

    current_best_location = fx_new_locations.argmin()
    best_location = new_locations[current_best_location]
    new_locations = np.delete(new_locations, current_best_location,0)
    RX = np.zeros((total_locations -1, 1)) 
    for i in range(total_locations-1):
        RX[i] = general_distance(new_locations[i], new_locations, d)

    prob = np.zeros((total_locations, 1))
    sum_RX = np.sum(RX)
    for i in range(total_locations-1):
        prob[i] = location_probability(sum_RX, RX[i],d)
    prob = prob.flatten()
    indexes = np.arange(total_locations)

    next_generation_selection = np.random.choice(indexes, n -1, p=prob, replace=False)

    X = np.zeros((n-1,d))
    for i in range(n-1):
        X[i] = next_generation_selection[i]
    X = np.vstack((best_location, X))
    print("current best location:\n",best_location)
    current_generation+=1

current best location:
 [ 0.17755457  1.30316099  2.28409427  0.81136814  0.54740178 -2.09190678
 -0.53879902 -2.34636781 -1.65148333 -1.47426815]


In [23]:
fx = y[4]
si = sparks_number(fx, y, m, ymax)
print("si: ",si)
print("fx: ", fx)
print("ymax: ",ymax)
sum = np.sum(ymax - y)
print("sum: ",sum)
spark =  (ymax - fx)/sum
print("ymax - y[0]/sum: ", spark)
print("m * ymax - fx/sum: ",m* spark)

si:  [8.]
fx:  [78.78535627]
ymax:  137.7006935494672
sum:  372.21326531167716
ymax - y[0]/sum:  [0.15828382]
m * ymax - fx/sum:  [7.91419097]


In [24]:
total_fireworks = np.sum(S)
total_fireworks

62.0

In [25]:
np.sort(np.random.choice(10,5, replace=False))

array([0, 2, 3, 6, 9])

In [26]:
xi = X[0]
gaussian_spark_location(xmin, xmax, d, xi)

array([ 0.2510135 ,  0.71211403,  2.48550116,  0.81136814,  0.54740178,
        0.82632003, -0.6562709 , -2.34636781, -2.17651554, -1.45354921])

In [27]:
new_locations[101]

array([ 2.34553008,  4.12494212,  3.54902903, -1.80762664, -2.53406342,
        0.98386032, -1.72025483,  2.81305928, -3.81820882,  3.11315142])

In [28]:
new_locations[69:]

array([[-4.3933076 , -4.13686969,  2.25651063,  2.89252411, -1.49747298,
         2.5280929 , -3.96846284,  0.61770753,  4.12472049, -3.00082018],
       [-0.07451786, -4.28446033, -0.67784064, -3.15397134, -1.49747298,
         2.5280929 , -0.49936262,  0.96096245,  4.12472049,  0.95268438],
       [-2.42562859,  3.98472167, -2.80451254,  4.70913443,  3.03418915,
        -0.50369144, -0.8681764 ,  0.02685239,  1.75607886,  0.33847144],
       [-3.18392892,  3.98472167, -3.6394176 ,  4.70913443, -3.93279476,
         4.16755403,  1.48097202, -0.22126592,  1.49513747,  0.13582478],
       [-2.42562859,  3.98472167, -2.80451254,  2.0969871 ,  1.03356054,
        -4.98196781,  1.47501557, -0.22126592,  1.75607886,  0.33847144],
       [-2.42562859,  3.98472167, -2.80451254,  4.70913443, -2.66508879,
         4.92427516,  1.73655342, -0.22126592,  1.75607886,  0.33847144],
       [ 0.5801002 ,  3.98472167,  0.04399791,  4.70913443, -2.66508879,
        -1.5510623 ,  1.73655342, -0.08234001

In [29]:
current_best_location

39

In [30]:
next_generation_selection[0]

110

In [31]:
print(indexes.shape)
print(prob)
next_gen = np.random.choice(indexes, total_locations -1, p=prob, replace=False)
print("NEXT GEN SIZE: ", len(next_gen))
print("total locations:", total_locations)

(147,)
[0.0064845  0.00666762 0.00712537 0.00713783 0.0074446  0.00640096
 0.00761116 0.0072078  0.00706027 0.00692186 0.00834668 0.00635642
 0.00701526 0.00701902 0.00710855 0.00658452 0.00659229 0.0066903
 0.00632743 0.00655961 0.00810629 0.00680676 0.0072056  0.00727458
 0.00666762 0.00712725 0.00706798 0.00818455 0.0065683  0.00720554
 0.0077361  0.00592482 0.00708871 0.00606834 0.00767567 0.00750678
 0.00665836 0.00751776 0.00817495 0.00728002 0.00677854 0.00681207
 0.00755402 0.00739377 0.0064377  0.00642752 0.00609655 0.0064541
 0.00658264 0.00749935 0.00634243 0.0073077  0.007374   0.00654522
 0.00738403 0.00721195 0.00744789 0.00719053 0.00651208 0.00714992
 0.0072078  0.00611356 0.00664934 0.00694335 0.00694474 0.00720252
 0.00669107 0.00677621 0.0064941  0.00685162 0.00613193 0.00658571
 0.006983   0.00693355 0.00666762 0.00632984 0.00628043 0.00607819
 0.00634629 0.00666762 0.00595531 0.00635205 0.00666934 0.00619935
 0.00650134 0.00666762 0.00689774 0.00694132 0.00687999 0